### Этот ноутбук создан для сентимент анализа

#### Импорты

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch
from torch import nn
from torch.nn import CrossEntropyLoss
from torch.nn.functional import softmax
import pandas as pd

In [ ]:
# Загружаем токенизатор и модель
tokenizer = AutoTokenizer.from_pretrained("fyaronskiy/ruRoberta-large-ru-go-emotions")
model = AutoModelForSequenceClassification.from_pretrained("fyaronskiy/ruRoberta-large-ru-go-emotions")

In [ ]:
# Переводим на видеокарту
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Перемещение модели на GPU

print(f"Используется устройство: {device}")
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))

Используется устройство: cuda
NVIDIA GeForce RTX 4090


In [ ]:
# Выбираем все эмоции для отображения
LABELS = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']
ID2LABEL = dict(enumerate(LABELS))

In [ ]:
# Загружаем датафрейм с текстами
df=pd.read_parquet('/root/my project ver 2/df_with_drugs_topic_modeled_ver2.parquet')
txt=df.iloc[15, 9] # возьмем текст для проверки
txt

'Чтоб взбодриться немного включаю поспишь потом\nВедь дела говорят типа сорри поспишь потом\nЯ набил тату с текстом спасибо за всё коллеги\nПрохожу эту жизнь как будто бы я Лилэгги\n\nПоезда самолёты такси метро и машины\nМоё сердце стучит и сжимается как пружина\nКак шумный клуб и безлюдный аэропорт\nОкей я пойду извини не люблю смол ток\n\nПалёный адидас и палёная пума а\nЭто мой путь это моя эстетика\nЛимитед эдишн эксклюзив и оригинал\nВсё хорошо но я это в рот ебал\nПалёный адидас и палёная пума а\nЭто мой путь это моя эстетика\nНе носишь подделки такой у тебя лайфстайл\nЯ копия фейк я Александр Паль\n\nЯ когда-нибудь сниму Александра Паля\nКогда я буду старый и он будет старый\nЭто будет типа фильм Побег из Шоушенка\nХочу чтобы лил дождь на половину бюджета\nСнова самолет тащит до дома\nДоброкола со льдом – это здорово!\nНо просыпаюсь в съемной квартире\nПонимаю это сон и вдыхаю бессилие\nПаленый адидас и паленая пума\nВ прокуренных лифтах палёные кнопки\nПаленая любовь и паленые

In [ ]:
# Функция для проверки на нескольких текстах вручную
def predict(text):  
    inputs = tokenizer(
        text,
        truncation=True,
        padding=True,
        add_special_tokens=True,
        max_length=128,
        return_tensors='pt'
    )
    

    for k in inputs:
        inputs[k] = inputs[k].to(device)
    
    model.to(device)  # на всякий случай

    with torch.no_grad():
        logits = model(**inputs).logits
        probas = torch.sigmoid(logits).squeeze(dim=0).tolist()

    probas = [round(p, 3) for p in probas]
    labels2probas = dict(zip(LABELS, probas))
    probas_dict_sorted = dict(sorted(labels2probas.items(), key=lambda x: x[1], reverse=True))
    return probas_dict_sorted

predict(txt)

{'gratitude': 0.911,
 'remorse': 0.063,
 'approval': 0.039,
 'realization': 0.036,
 'neutral': 0.031,
 'sadness': 0.03,
 'joy': 0.024,
 'disappointment': 0.022,
 'disapproval': 0.014,
 'excitement': 0.012,
 'love': 0.011,
 'admiration': 0.009,
 'amusement': 0.008,
 'annoyance': 0.008,
 'relief': 0.008,
 'caring': 0.006,
 'confusion': 0.006,
 'desire': 0.005,
 'embarrassment': 0.005,
 'optimism': 0.005,
 'pride': 0.004,
 'surprise': 0.004,
 'anger': 0.003,
 'disgust': 0.003,
 'fear': 0.003,
 'grief': 0.003,
 'nervousness': 0.003,
 'curiosity': 0.002}

Разметим все тексты и выведем топ-5 эмоций на каждый текст

In [30]:
# Функция предсказания топ-5 эмоций с вероятностями
def predict_top5_emotions(text):
    inputs = tokenizer(
        text,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="pt"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        logits = model(**inputs).logits
        probs = torch.sigmoid(logits).squeeze(0).tolist()

    # Сопоставление эмоций с вероятностями
    emotions = dict(zip(LABELS, [round(p, 3) for p in probs]))

    # Сортировка и взятие топ-5
    top5 = dict(sorted(emotions.items(), key=lambda x: x[1], reverse=True)[:5])
    return top5

In [ ]:
# Размечаем и смотрим
df['top5_emotions'] = df['Текст трека'].apply(predict_top5_emotions)
df.head()

,Исполнитель,Трек,Собран,Жанр,Доп. информация,Фит,Совместный релиз,Альбом,Год выхода,Текст трека,...,Части текста,Обработанные части,Очищенные леммы,predicted_label,confidence,prob_class_1,topic,topic_keywords,topic_probability,top5_emotions
original_index,,,,,,,,,,,,,,,,,,,,,
https://music.yandex.ru/album/21509172/track/101776208,SaNni,Где бывают боги,1.0,русский рэп,Нет,Нет,Несовместный релиз,Сингл,2019.0,"Это зачем?\nЧтоб боялись\nКого боятся, тот и г...",...,"[Это зачем?\nЧтоб боялись\nКого боятся, тот и ...","[[это, зачем, ?, чтобы, бояться, кто, бояться,...","[[это, бояться, бояться, главний, слушать, схо...",1,0.983454,0.983454,-1,None,0.0,"{'neutral': 0.658, 'curiosity': 0.298, 'annoya..."
https://music.yandex.ru/album/20685194/track/99172841,SaNni,Моя,1.0,русский рэп,Нет,Нет,Alett,Сингл,2022.0,"Снова заберу тебя с собой, будто под водой\nНе...",...,"[Снова заберу тебя с собой, будто под водой\nН...","[[снова, забрать, ты, с, себя, ,, будто, под, ...","[[снова, забрать, вода, хватать, кислород, зна...",1,0.958730,0.958730,-1,None,0.0,"{'love': 0.77, 'neutral': 0.299, 'caring': 0.0..."
https://music.yandex.ru/album/25088929/track/111941871,Klarko,Этому сну не стать вещим,1.0,русский рэп,Нет,Нет,Несовместный релиз,Сингл,2023.0,Пятьсот килоджоулей я сжег романтично\nДымит d...,...,[Пятьсот килоджоулей я сжег романтично\nДымит ...,"[[пятьсот, килоджоуль, я, сжечь, романтичный, ...","[[пятьсот, килоджоуль, сжечь, романтичный, Дым...",1,0.983501,0.983501,-1,None,0.0,"{'neutral': 0.813, 'approval': 0.035, 'joy': 0..."
https://music.yandex.ru/album/9269095/track/60155140,АЛМО,Сонный паралич,1.0,русский рэп,Нет,Нет,Несовместный релиз,Сингл,2019.0,Руки в волосах твоих пропахли керосином\nЯ пок...,...,[Руки в волосах твоих пропахли керосином\nЯ по...,"[[рука, в, волос, твой, пропахнуть, керосин, я...","[[рука, волос, твой, пропахнуть, керосин, поки...",1,0.766326,0.766326,2,"всё, нам, мир, тебе, лишь, жизнь, свет, небо, ...",1.0,"{'neutral': 0.282, 'joy': 0.123, 'excitement':..."
https://music.yandex.ru/album/4229094/track/34362334,ИЧИ,Обратно,1.0,русский рэп,Нет,"ATL, ИЧИ",Несовместный релиз,Лимб,2017.0,"Мама, роди меня обратно\n\nЯ пожалел об этом т...",...,"[Мама, роди меня обратно, Я пожалел об этом ты...","[[мама, ,, родить, я, обратно], [я, жалеть, о,...","[[мама, родить, обратно], [жалеть, это, тысяче...",1,0.988983,0.988983,-1,None,0.0,"{'sadness': 0.456, 'remorse': 0.325, 'disappoi..."


In [ ]:
# Сохраняем
df.to_parquet('/root/my project ver 2/df_with_drugs_topic_modeled_sentiment.parquet')